# Encoder Decoder

## Load data, functions

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import std
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pywt
import matrixprofile as mp

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead):
    X, y = list(), list()
    example_count = int((len(sequence)/step_interval))
    for i in range(example_count):
        # find the end of this pattern
        end_ix = (i*step_interval) + n_steps_in
        out_start_ix = end_ix + n_step_lookahead -1
        out_end_ix = end_ix + n_steps_out + n_step_lookahead -1
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[(i*step_interval):end_ix], sequence[out_start_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
#To demonstrate above function
sequence = range(0,13)
n_steps_in = 1
n_steps_in = 5
n_steps_out =1
step_interval =1
n_step_lookahead=5
split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead)

(array([[0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5],
        [2, 3, 4, 5, 6],
        [3, 4, 5, 6, 7]]),
 array([[ 9],
        [10],
        [11],
        [12]]))

In [4]:
percentile_data = pd.read_csv (r'C:\Users/conal/Desktop/MCM/Practicum - Copy/data/block gas price percentile data.csv', header=0)
percentile_data['datetime'] = pd.to_datetime(percentile_data['block_timestamp'], format = '%Y-%m-%d %H:%M:%S UTC')

percentile_data = percentile_data.sort_values(by='datetime',ascending=False)
percentile_data = percentile_data.set_index('datetime')
percentile_data = percentile_data.resample('5T').mean()
percentile_data = percentile_data/1000000000

In [5]:
usage_data = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\ETH,gas,usage merged 11-26 to 05-26.csv', header=0)
usage_data['datetime'] = pd.to_datetime(usage_data['datetime'], format = '%Y-%m-%d %H:%M:%S')
usage_data = usage_data.set_index('datetime')

usage_data = usage_data.squeeze()
usage_data = usage_data.astype('float')
usage_data = usage_data.resample('5T').mean()

In [6]:
usage_data2 = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\Contract counts 2021-11-26 to 2022-05-26.csv', header=0, index_col=0)
usage_data2['datetime'] = pd.to_datetime(usage_data2['block_timestamp'], format = '%Y-%m-%d %H:%M:%S') 
usage_data2 = usage_data2.set_index('datetime')
usage_data2 = usage_data2.drop(['block_timestamp'], axis=1)
usage_data2 = usage_data2.squeeze()
usage_data2 = usage_data2.astype('float')
usage_data2 = usage_data2.resample('5T').sum()

In [7]:
data = usage_data.merge(percentile_data, left_index=True, right_index=True)
data = data.merge(usage_data2, left_index=True, right_index=True)

Load data, datetime to index, downsample with left edge label, convert wei to gwei

In [8]:
def generate_training_val_examples(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    #we are only lookign to forecast the min gas price
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))

    
    return X_train, y_train, X_val, y_val, scaler



In [30]:
def LSTM_model():
    model = Sequential()
    model.add(LSTM(100, activation='tanh', input_shape=(n_steps_in, len(inputs))))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(100, activation='tanh', return_sequences=True))
    model.add(TimeDistributed(Dense(len(inputs))))
    model.compile(optimizer='adam', loss='mse')
    return model

In [9]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


In [10]:
def descale_y_retrun_metrics(yhat, y_val2):
    #We will use validation data that has not had outleirs limited, will be a different min/max scaler as such
    dict_indexes=[]
    dict_dfs=[]
    for j in range(0, n_steps_out):
        RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
        for i in range(0, len(inputs)):  
            pred_descaled= (scaler.inverse_transform(yhat[:,j:j+1,:].reshape(yhat.shape[0], yhat.shape[2])))[:, i:i+1]
            groud_truth_descaled= ((scaler2.inverse_transform(y_val2[:,j:j+1,:].reshape(y_val2.shape[0], y_val2.shape[2]))))[:, i:i+1]
            RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
            MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
            MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
            MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
            R2 = r2_score(groud_truth_descaled, pred_descaled)
            RMSE_list.append(RMSE)
            MAE_list.append(MAE)
            MAPE_list.append(MAPE)
            R2_list.append(R2)
            MSE_list.append(MSE)
        metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=inputs)
        dict_dfs.append(metrics_df)
        dict_indexes.append('Lookahead' +str(j))
    metrics_dict = dict(zip(dict_indexes, dict_dfs))
    return metrics_dict

## Lets optimize loss function towards prediction of all variables

In [167]:
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032
n_steps_out = 5
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples(data)
    

    
    
    model = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save(('Encoder_Decoder_2/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)
    
    training_metrics_dicts.append(descale_y_retrun_metrics(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics(yhat_val, y_val))
    
    
    

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
107/107 [==============================] - 20s 148ms/step - loss: 0.5415 - val_loss: 0.3002
Epoch 2/15
107/107 [==============================] - 15s 140ms/step - loss: 0.4914 - val_loss: 0.3033
Epoch 3/15
107/107 [==============================] - 15s 139ms/step - loss: 0.4813 - val_loss: 0.3092
Epoch 4/15
107/107 [==============================] - 15s 140ms/step - loss: 0.4652 - val_loss: 0.3012
Epoch 5/15
107/107 [==============================] - 16s 149ms/step - loss: 0.4586 - val_loss: 0.3186
Epoch 6/15
107/107 [==============================] - 15s 144ms/step - loss: 0.4530 - val_loss: 0.4579
Epoch 7/15
107/107 [==============================] - 15s 144ms/step - loss: 0.4491 - val_loss: 0.3204
Epoch 8/15
107/107 [==============================] - 15s 143ms/step - loss: 0.4327 - val_loss: 0.3162
Epoch 9/15
107/107 [==============================] - 15s 144ms/step - loss: 0.4356 - val_loss: 0.3104
Epoch 10/15
107/107 [==============================] - 15s 143ms/step - l

INFO:tensorflow:Assets written to: Encoder_Decoder/Month0_pears_step_lookahead\assets


INFO:tensorflow:Assets written to: Encoder_Decoder/Month0_pears_step_lookahead\assets


46/46 [==============================] - 3s 51ms/step


...

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
108/108 [==============================] - 22s 165ms/step - loss: 0.4163 - val_loss: 0.3550
Epoch 2/15
108/108 [==============================] - 16s 152ms/step - loss: 0.3930 - val_loss: 0.3475
Epoch 3/15
108/108 [==============================] - 16s 151ms/step - loss: 0.3842 - val_loss: 0.3488
Epoch 4/15
108/108 [==============================] - 16s 150ms/step - loss: 0.3809 - val_loss: 0.3460
Epoch 5/15
108/108 [==============================] - 16s 150ms/step - loss: 0.3774 - val_loss: 0.3479
Epoch 6/15
108/108 [==============================] - 16s 149ms/step - loss: 0.3743 - val_loss: 0.3459
Epoch 7/15
108/108 [==============================] - 15s 143ms/step - loss: 0.3729 - val_loss: 0.3482
Epoch 8/15
108/108 [==============================] - 16s 144ms/step - loss: 0.3694 - val_loss: 0.3504
Epoch 9/15
108/108 [==============================] - 15s 144ms/step - loss: 0.3720 - val_loss: 0.3481
Epoch 10/15
108/108 [==============================] - 15s 143ms/step - l

INFO:tensorflow:Assets written to: Encoder_Decoder/Month4_pears_step_lookahead\assets


INFO:tensorflow:Assets written to: Encoder_Decoder/Month4_pears_step_lookahead\assets


46/46 [==============================] - 2s 50ms/step


In [8]:
np.load('Encoder_Decoder_2/validation_metric_dict_1.npy', allow_pickle=True)

array([[[1.68098673e+01, 2.82571638e+02, 8.64928467e+00, 1.19404758e-01,
         5.05659610e-01],
        [1.89262528e+01, 3.58203046e+02, 1.08748015e+01, 1.50478888e-01,
         3.73165120e-01],
        [1.99322571e+01, 3.97294873e+02, 1.21897020e+01, 1.70127489e-01,
         3.04794814e-01],
        [2.12319241e+01, 4.50794603e+02, 1.29565851e+01, 1.82262335e-01,
         2.11900081e-01],
        [2.23455285e+01, 4.99322645e+02, 1.34147264e+01, 1.89091133e-01,
         1.27186224e-01]],

       [[3.17902620e+01, 1.01062076e+03, 1.78014682e+01, 1.15707586e-01,
         7.35022476e-01],
        [4.05445919e+01, 1.64386393e+03, 2.33814681e+01, 1.55269704e-01,
         5.69041417e-01],
        [4.41085812e+01, 1.94556693e+03, 2.61993724e+01, 1.76175617e-01,
         4.90110759e-01],
        [4.62554880e+01, 2.13957017e+03, 2.78675563e+01, 1.89776721e-01,
         4.39133457e-01],
        [4.73942896e+01, 2.24621869e+03, 2.92279299e+01, 2.01785489e-01,
         4.11122340e-01]],

      

In [191]:
average_keys = list(valdiation_metrics_dicts[0].keys())

In [263]:
#Final lookahead for all months
(valdiation_metrics_dicts[0]['Lookahead4']+valdiation_metrics_dicts[1]['Lookahead4']+valdiation_metrics_dicts[2]['Lookahead4']+valdiation_metrics_dicts[3]['Lookahead4']+valdiation_metrics_dicts[4]['Lookahead4'])/5

,RMSE,MSE,MAE,MAPE,R2
min_gas_price,2.955788e+01,1.088852e+03,1.703250e+01,0.235236,0.304757
block_gas_5th_percentile,3.478283e+01,1.604899e+03,1.716701e+01,0.231692,0.265231
block_gas_95th_percentile,7.806099e+01,8.597283e+03,2.878426e+01,0.269449,0.005841
gas_used,2.023481e+06,4.105746e+12,1.606715e+06,0.094682,-0.014156
base_fee_per_gas,2.566189e+01,8.275509e+02,1.626360e+01,0.230212,0.397588
transaction_count,3.622706e+01,1.313428e+03,2.775937e+01,0.137649,-0.044208
size,1.734421e+04,3.114874e+08,1.327472e+04,0.139229,-0.026644
Open,9.572247e+01,1.498083e+04,8.002370e+01,0.030349,-0.459802
contracts,3.699251e+01,1.541857e+03,2.199122e+01,0.551590,0.014958


Encoder-Decoder shows improved performance over a simple LSTM when predicting multivariate data.

In [269]:
averaged_dfs = []
average_keys = list(valdiation_metrics_dicts[0].keys())
for j in average_keys:
    average_df = verage_df=[]
    for i in range(0, len(valdiation_metrics_dicts)):
        average_df.append(valdiation_metrics_dicts[i][j])
    averaged_dfs.append((sum(average_df))/5)   
averaged_metrics_dict = dict(zip(average_keys, averaged_dfs))

In [270]:
averaged_metrics_dict

{'Lookahead0':                                    RMSE           MSE           MAE      MAPE  \
 min_gas_price              2.161757e+01  5.916021e+02  1.143324e+01  0.162173   
 block_gas_5th_percentile   2.827227e+01  1.122858e+03  1.171577e+01  0.160988   
 block_gas_95th_percentile  7.129544e+01  7.546303e+03  2.070955e+01  0.193550   
 gas_used                   1.985275e+06  3.951620e+12  1.569293e+06  0.092002   
 base_fee_per_gas           1.674487e+01  3.438127e+02  1.074580e+01  0.158522   
 transaction_count          3.498671e+01  1.224803e+03  2.666822e+01  0.131644   
 size                       1.634182e+04  2.749926e+08  1.257136e+04  0.131681   
 Open                       1.059465e+02  1.922064e+04  9.091249e+01  0.034346   
 contracts                  3.539016e+01  1.395937e+03  2.092365e+01  0.510994   
 
                                  R2  
 min_gas_price              0.627645  
 block_gas_5th_percentile   0.520705  
 block_gas_95th_percentile  0.146025  
 gas_use

## Lets optimize for min gas price only

In [11]:
def generate_training_val_examples_univariate_output(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM, filter output to only the first input
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))[:,:,:1]
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))[:,:,:1]

    
    return X_train, y_train, X_val, y_val, scaler

In [12]:
def descale_y_retrun_metrics_univariate_y(yhat, y_val2):
    #reverts standard scaling, returns dictionary of metrics for single output, for all lookaheads
    dict_indexes=[]
    dict_dfs=[]
    RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
    for j in range(0, n_steps_out):
        pred_descaled= (scaler.inverse_transform(yhat[:, j:j+1, :len(inputs)].reshape(yhat.shape[0], len(inputs))))[:,:1]
        groud_truth_descaled= (scaler.inverse_transform(array([y_val2[ :, j:j+1,0].reshape(y_val2.shape[0])]*len(inputs)).transpose()))[:,:1]
        RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
        MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
        MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
        MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
        R2 = r2_score(groud_truth_descaled, pred_descaled)
        RMSE_list.append(RMSE)
        MAE_list.append(MAE)
        MAPE_list.append(MAPE)
        R2_list.append(R2)
        MSE_list.append(MSE)
    metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=range(1, (n_steps_out+1)))

 
    return metrics_df

In [40]:
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032
n_steps_out = 5
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    

    
    
    model = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save(('Encoder_Decoder/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)
    
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))

    pd.DataFrame(train_loss_list).to_csv('Encoder_Decoder/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('Encoder_Decoder/val_loss.csv')

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
107/107 [==============================] - 22s 166ms/step - loss: 0.3511 - val_loss: 0.0493
Epoch 2/15
107/107 [==============================] - 16s 150ms/step - loss: 0.3114 - val_loss: 0.0646
Epoch 3/15
107/107 [==============================] - 17s 158ms/step - loss: 0.3002 - val_loss: 0.0461
Epoch 4/15
107/107 [==============================] - 16s 148ms/step - loss: 0.2813 - val_loss: 0.0494
Epoch 5/15
107/107 [==============================] - 16s 148ms/step - loss: 0.2684 - val_loss: 0.0777
Epoch 6/15
107/107 [==============================] - 16s 146ms/step - loss: 0.2599 - val_loss: 0.0882
Epoch 7/15
107/107 [==============================] - 15s 145ms/step - loss: 0.2520 - val_loss: 0.0620
Epoch 8/15
107/107 [==============================] - 16s 146ms/step - loss: 0.2385 - val_loss: 0.0651
Epoch 9/15
107/107 [==============================] - 15s 144ms/step - loss: 0.2359 - val_loss: 0.0708
Epoch 10/15
107/107 [==============================] - 15s 145ms/step - l

INFO:tensorflow:Assets written to: Encoder_Decoder/Month0\assets


INFO:tensorflow:Assets written to: Encoder_Decoder/Month0\assets


46/46 [==============================] - 3s 48ms/step


...

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
108/108 [==============================] - 22s 166ms/step - loss: 0.1119 - val_loss: 0.0371
Epoch 2/15
108/108 [==============================] - 17s 159ms/step - loss: 0.0687 - val_loss: 0.0356
Epoch 3/15
108/108 [==============================] - 17s 155ms/step - loss: 0.0621 - val_loss: 0.0363
Epoch 4/15
108/108 [==============================] - 16s 152ms/step - loss: 0.0615 - val_loss: 0.0342
Epoch 5/15
108/108 [==============================] - 17s 158ms/step - loss: 0.0563 - val_loss: 0.0346
Epoch 6/15
108/108 [==============================] - 17s 158ms/step - loss: 0.0541 - val_loss: 0.0354
Epoch 7/15
108/108 [==============================] - 16s 152ms/step - loss: 0.0567 - val_loss: 0.0378
Epoch 8/15
108/108 [==============================] - 16s 153ms/step - loss: 0.0537 - val_loss: 0.0363
Epoch 9/15
108/108 [==============================] - 16s 148ms/step - loss: 0.0521 - val_loss: 0.0387
Epoch 10/15
108/108 [==============================] - 16s 151ms/step - l

INFO:tensorflow:Assets written to: Encoder_Decoder/Month4\assets


INFO:tensorflow:Assets written to: Encoder_Decoder/Month4\assets


46/46 [==============================] - 2s 53ms/step


In [44]:
sum(valdiation_metrics_dicts)/5

,RMSE,MSE,MAE,MAPE,R2
1,21.304548,555.266234,10.670038,0.145905,0.623420
2,25.494286,787.138785,13.549184,0.182654,0.463588
3,27.167672,891.550477,14.850080,0.199498,0.389870
4,28.280883,964.446965,15.688489,0.212589,0.336666
5,29.071801,1013.453827,16.337408,0.221948,0.292530


In [46]:
sum(training_metrics_dicts)/5

,RMSE,MSE,MAE,MAPE,R2
1,24.776080,650.818995,10.074590,0.135005,0.696787
2,28.278432,861.197492,12.848631,0.170483,0.616118
3,28.642705,907.210278,13.570277,0.179138,0.605765
4,29.274516,956.460390,14.130286,0.188039,0.591803
5,30.087603,1024.946407,14.661874,0.194577,0.572933
